In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

In [2]:
print(tf.__version__)

2.11.0


In [3]:
petrol_cons = pd.read_csv('/data/petrol_consumption.csv')

FileNotFoundError: [Errno 2] File /data/petrol_consumption.csv does not exist: '/data/petrol_consumption.csv'

In [ ]:
petrol_cons.head()

In [ ]:
X = petrol_cons.iloc[:, 0:4].values
y = petrol_cons.iloc[:, 4].values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Model

In [ ]:
input_layer = Input(shape=(X.shape[1],))
dense_layer_1 = Dense(100, activation='relu')(input_layer)
dense_layer_2 = Dense(50, activation='relu')(dense_layer_1)
dense_layer_3 = Dense(25, activation='relu')(dense_layer_2)
output = Dense(1)(dense_layer_3)

model_tf = Model(inputs=input_layer, outputs=output)
model_tf.compile(loss="mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])

In [ ]:
print(model_tf.summary())

In [ ]:
history = model_tf.fit(X_train, y_train, batch_size=8, epochs=30, verbose=1, validation_split=0.2)

In [ ]:
history = model_tf.fit(X_train, y_train, batch_size=2, epochs=50, verbose=1, validation_split=0.2)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

pred_train = model_tf.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train)))

pred = model_tf.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred)))

In [ ]:
cols = ['Petrol_tax','Average_income','Paved_Highways','Population_Driver_licence(%)']

In [ ]:
#Model function
def model(algorithm,dtrain_X,dtrain_Y,dtest_X,dtest_Y,cols=None):

    #algorithm.fit(dtrain_X,dtrain_Y)
    predictions = algorithm.predict(dtest_X)
    prediction_probabilities = algorithm.predict(dtest_X)
    print (algorithm)
    
    return predictions,prediction_probabilities,algorithm

In [ ]:
y_pred,y_prob,model_obj=model(model_tf,X_train,y_train,X_test,y_test)

In [ ]:
from mosaicml import *
from mosaicml.constants import MLModelFlavours

In [ ]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = model.predict(data)
    #prediction = pd.Series(model.predict(data))
    #print (prediction)
    return prediction
    #return prediction.tolist()

In [ ]:
petrol_cons.head()

In [ ]:
X_temp = pd.DataFrame(X,columns=cols)

In [ ]:
import requests
req = requests.Request()
req.json = {"payload":X_temp.head(1).to_json(orient = 'records')}

In [ ]:
req.json['payload']

In [ ]:
sch = generate_schema(score,(model_obj, req),X_temp)
print(sch)

In [ ]:
register_model(model_obj, score, name="CarEvaluation_RegressionTENSORFLOW", 
                description="Classify Car basis the feature",
                flavour=MLModelFlavours.tensorflow, schema=sch,
                y_true=y_test, y_pred = y_pred, #y_pred=pd.Series(y_pred), 
                prob=y_pred, features=cols, 
               labels=["Petrol_Consumption"], 
               init_script="" ,
                model_type="regression", 
                input_type="json", explain_ai=False, x_train=X_train, x_test=X_test, 
                y_train=y_train, y_test=y_test,
                feature_names=cols,feature_ids=cols,
                target_names=["Petrol_Consumption"], 
                kyd=False, kyd_score =False)
print("Registering model")